# 객체감지

In [2]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
result = model('YellowLabradorLooking_new.jpg')
result


image 1/1 c:\githome\data_anal_DL\YellowLabradorLooking_new.jpg: 544x640 1 dog, 53.3ms
Speed: 10.5ms preprocess, 53.3ms inference, 3.6ms postprocess per image at shape (1, 3, 544, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [4]:
# 바운딩 박스, 라벨 출력
result = model('YellowLabradorLooking_new.jpg')
result[0].boxes


image 1/1 c:\githome\data_anal_DL\YellowLabradorLooking_new.jpg: 544x640 1 dog, 21.3ms
Speed: 4.5ms preprocess, 21.3ms inference, 6.1ms postprocess per image at shape (1, 3, 544, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([16.], device='cuda:0')
conf: tensor([0.6653], device='cuda:0')
data: tensor([[ 51.3454,  33.1425, 629.2720, 555.0193,   0.6653,  16.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (577, 700)
shape: torch.Size([1, 6])
xywh: tensor([[340.3087, 294.0809, 577.9266, 521.8768]], device='cuda:0')
xywhn: tensor([[0.4862, 0.5097, 0.8256, 0.9045]], device='cuda:0')
xyxy: tensor([[ 51.3454,  33.1425, 629.2720, 555.0193]], device='cuda:0')
xyxyn: tensor([[0.0734, 0.0574, 0.8990, 0.9619]], device='cuda:0')

In [5]:
import cv2
img = cv2.imread('YellowLabradorLooking_new.jpg')

In [7]:
for box in result[0].boxes:
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    # 신뢰도 점수
    conf = float(box.conf[0])
    
    # 바운딩 박스
    coords = box.xyxy[0].tolist()
    x1, y1, x2, y2 = map(int, coords)
    cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.putText(img, f'{class_name} {conf:.4f}', (x1,y1-10), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0))
    
cv2.imshow('yolov8 object detection', img)
cv2.waitKey()
cv2.destroyAllWindows()

# segmentation

In [10]:
model = YOLO('yolov8n-seg.pt')
result = model('road.png')


image 1/1 c:\githome\data_anal_DL\road.png: 448x640 2 persons, 2 motorcycles, 24.4ms
Speed: 2.9ms preprocess, 24.4ms inference, 9.4ms postprocess per image at shape (1, 3, 448, 640)


In [16]:
import numpy as np
img_new = cv2.imread('road.png')
overlay = img_new.copy()  # 마스크용

for box, mask in zip(result[0].boxes, result[0].masks):
    
    polygon = mask.xy[0].astype(np.int32)
    cv2.fillPoly(overlay, [polygon], (0,255,0))
    
    class_id = box.cls[0]
    class_name = model.names[class_id.item()]
    
    # 신뢰도 점수
    conf = float(box.conf[0])
    
    # 바운딩 박스
    coords = box.xyxy[0].tolist()
    x1, y1, x2, y2 = map(int, coords)
    cv2.rectangle(img_new, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.putText(img_new, f'{class_name} {conf:.4f}', (x1,y1-10), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0))
    
# img + overlay 겹쳐서 출력
alpha = 0.5
final_img = cv2.addWeighted(overlay, alpha, img_new, 1-alpha, 0)

# 최종 출력
cv2.imshow('yolov8 object detection', final_img)
cv2.waitKey()
cv2.destroyAllWindows()

# Pose

In [28]:
pose_model = YOLO('yolov8n-pose.pt')
pose_result = pose_model('people.png')
img = cv2.imread('people.png')

kp = pose_result[0].keypoints

for man_kp in kp:
    for x, y in man_kp.xy[0]:
        if x>0 and y>0:
            cv2.circle(img, (int(x), int(y)), 3, (255,0,0), -1)
            
cv2.imshow('yolov8 pose estimation', img)
cv2.waitKey()
cv2.destroyAllWindows()


image 1/1 c:\githome\data_anal_DL\people.png: 640x480 4 persons, 62.3ms
Speed: 3.7ms preprocess, 62.3ms inference, 9.9ms postprocess per image at shape (1, 3, 640, 480)
